# Import libraries

In [1]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
import ot
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

In [3]:
#-- Set base directory and data directory path --#
basePath   = './'
dataPath   = 'Data/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

# Functions

In [4]:
%run centralFunctions.ipynb

# Loading Data

In [5]:
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

In [6]:
bkg_data = dataDict['bkg']['Particles'][:,:,0:3]
sig_data = {}

for alias in sigAliasList:
  sig_data[alias] = dataDict[alias]['Particles'][:,:,0:3]

# rNN Classification

In [7]:
nEvents = 500
random_state = Generator(PCG64(123))
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [13]:
total_event_pT = {}

total_event_pT['bkg'] = np.sum(bkg_data[:, :, 0], axis=1)

for alias in sigAliasList:
    total_event_pT[alias] = np.sum(sig_data[alias][:,:,0], axis=1)

pTrange = [0,50,100,150,200,500,1000]
radius_list = [3,3.5, 4,4.5, 5,5.5,6,6.5,7,7.5, 8,9, 10]
avg_aucs = []
std_aucs = []
avg_radii = []
std_radii = []
for i in range(0, len(pTrange)-1):
    lower_bound = pTrange[i]
    upper_bound = pTrange[i+1]
    
    filtered_events = {}
    mask = (total_event_pT['bkg'] >= lower_bound) & (total_event_pT['bkg'] <= upper_bound)
    
    filtered_events['bkg'] = randomDataSample(bkg_data[mask],nEvents,random_state)
    np.random.seed(i)
    permutation = np.random.permutation(nEvents*2)
    
    for alias in sigAliasList:
        mask = (total_event_pT[alias] >= lower_bound) & (total_event_pT[alias] <= upper_bound)
        filtered_events[alias] = randomDataSample(sig_data[alias][mask],nEvents,random_state)
        
        event_list = np.concatenate((filtered_events['bkg'],filtered_events[alias]))
        event_labels = np.asarray([0] * nEvents + [1] * nEvents)
        event_list = event_list[permutation]
        event_labels = event_labels[permutation]
        
        distance_matrix = calcOTDistance(event_list, event_list, OTSCHEME, '2D', Matrix = True)
        print(np.mean(distance_matrix))
        
        avg_auc, std_auc, avg_radius, std_radius = rNN_cross_validation(distance_matrix, event_labels, radius_list, k_fold=5)
        print(avg_auc, std_auc, avg_radius, std_radius)
        avg_aucs.append(avg_auc)
        std_aucs.append(std_auc)
        avg_radii.append(avg_radius)
        std_radii.append(std_radius)

100%|██████████| 1000000/1000000 [01:15<00:00, 13310.96it/s]


5.591899946925802


Fitting Models: 100%|██████████| 13/13 [00:00<00:00, 460.94it/s]


0.55805499090703 0.03218034212899328 4.3 1.6309506430300091


100%|██████████| 1000000/1000000 [01:13<00:00, 13693.32it/s]


...

0.5508315771030226 0.030101439225498707 4.6 1.624807680927192


100%|██████████| 1000000/1000000 [01:14<00:00, 13412.53it/s]


5.172685012204326


Fitting Models:   0%|          | 0/13 [00:00<?, ?it/s]


ValueError: No neighbors found for test samples array([85]), you can try using larger radius, giving a label for outliers, or considering removing them from your dataset.

In [9]:
print(avg_aucs)
print(std_aucs)

[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287, 0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318, 0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865, 0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944, 0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541, 0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]
[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139, 0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385, 0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753, 0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636, 0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634, 0.02230500501928155, 0.021238964288079334, 0.0341775808408001, 0.015364811569643429]


In [10]:
grouped_data_1 = [avg_aucs[i:i+4] for i in range(0, len(avg_aucs), 4)]
grouped_data_2 = [std_aucs[i:i+4] for i in range(0, len(std_aucs), 4)]
print(grouped_data_1)
print(grouped_data_2)

[[0.6193241561554373, 0.5560362022323223, 0.5752272736176252, 0.6840947062389287], [0.797380753997553, 0.6761395554738507, 0.607118567366921, 0.7468665866068318], [0.7011452113981134, 0.5667354772174112, 0.6015605971417531, 0.6883180348051865], [0.6251299810860171, 0.5761864516416402, 0.6665967908890511, 0.6981814488486944], [0.6258593862764383, 0.5841808994695518, 0.6885509588655917, 0.6166094819906541], [0.798913933102407, 0.5829378691399614, 0.6202817253317441, 0.5284963612145445]]
[[0.013909554559565948, 0.026013491872497616, 0.037555171735308014, 0.03150385066000139], [0.008993921054521814, 0.035688653156475125, 0.026939186162531433, 0.021490142570251385], [0.005785647854299646, 0.04953517731908581, 0.03723336061553597, 0.015572400289487753], [0.020683095471606176, 0.02572489625664325, 0.021584085318690237, 0.019362046157130636], [0.024167621160884223, 0.006719364053971755, 0.020427488648409705, 0.020159541926277634], [0.02230500501928155, 0.021238964288079334, 0.0341775808408001,